<a href="https://colab.research.google.com/github/misrori/signals/blob/main/rsi_strategy_tester_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install goldhand


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from goldhand import *
from tqdm import tqdm
from IPython.display import display
import numpy as np
import pandas_ta as ta
import warnings
pd.set_option('display.max_rows', 70)
warnings.filterwarnings('ignore')

tw=Tw()



# RSI

In [4]:
def rsi_strategy(data, buy_threshold = 30, sell_threshold = 70):

    in_trade = False  # Flag to track if already in a trade
    trade_id = 1
    all_trades = []

    temp_trade = {}

    for i in range(1, len(data)):
        # Check if not already in a trade
        if not in_trade:
            # Generate buy signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] < buy_threshold:

                if i == (len(data) -1):
                    temp_trade['buy_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('buy_')))
                else:
                    temp_trade['buy_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('buy_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'open'
                in_trade = True  # Set flag to indicate in a trade
        else:
            # Generate sell signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] > sell_threshold:
                if i == (len(data) -1):
                    temp_trade['sell_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('sell_')))
                else:
                    temp_trade['sell_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('sell_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'closed'


                # calculate results
                temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
                temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days

                in_trade = False  # Reset flag to indicate not in a trade
                trade_id +=1
                all_trades.append(temp_trade)
                temp_trade = {}

    if temp_trade:
        temp_trade['sell_price'] = data['close'][i]
        temp_trade['trade_id'] = trade_id
        temp_trade['sell_date'] = data['date'][i]

        temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
        temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days
        all_trades.append(temp_trade)

    res_df = pd.DataFrame(all_trades)

    # change orders
    all_col = res_df.columns.tolist()
    first = ['result', 'buy_price', 'sell_price', 'buy_date', 'sell_date', 'days_in_trade']
    first.extend([x for x in all_col if x not in first])
    res_df = res_df[first]
    return(res_df)


# Apply for top 20 stocks

In [5]:
all_trades = []
all_trades_summaries=[]

for ticker in tqdm(tw.stock['name'][0:20]):
    try:
        # get data
        data = GoldHand(ticker).df

        # backtest
        backtest = Backtest( data, rsi_strategy, plot_title=tw.get_plotly_title(ticker), buy_threshold=28, sell_threshold=75)

        # results of backtest
        trades = backtest.trades
        trades_summary= backtest.trades_summary

        # append results
        all_trades.append(trades)
        all_trades_summaries.append(trades_summary)

    except:
      pass


tradesdf = pd.concat(all_trades, ignore_index=True)
trades_summarydf = pd.DataFrame(all_trades_summaries)



  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:05<00:00,  3.67it/s]


In [8]:
trades_summarydf.to_csv('trades_summary.csv', index=False)
tradesdf.to_csv('trades.csv', index=False)

In [11]:
! git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   rsi_strategy_tester_stock.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	trades.csv
	trades_summary.csv

no changes added to commit (use "git add" and/or "git commit -a")


In [12]:
git add .


SyntaxError: invalid syntax (677284323.py, line 1)

In [6]:
trades_summarydf

,ticker,number_of_trades,win_ratio(%),average_res(%),average_trade_len(days),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,...,looser_trades_mean,looser_trades_median,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
0,AAPL,9,77.78,15.10,225.0,7.90,3.231036,23.56 # 31.91 # -1.58 # -11.45 # 7.72 # 30.13 ...,23.56 # 31.91 # 7.72 # 30.13 # 6.63 # 41.03 # 7.9,21.27,...,-6.51,-6.51,179.0,7,2,41.03,-11.45,2006-02-06,28,75
1,MSFT,9,77.78,8.57,243.0,14.39,1.928270,5.51 # 14.39 # -21.79 # -5.34 # 14.67 # 4.79 #...,5.51 # 14.39 # 14.67 # 4.79 # 15.39 # 23.4 # 2...,14.90,...,-13.57,-13.57,170.0,7,2,26.13,-21.79,2006-05-01,28,75
2,GOOG,12,83.33,13.89,191.0,13.20,4.505082,19.98 # 6.74 # 5.18 # -1.7 # 10.02 # 23.01 # 2...,19.98 # 6.74 # 5.18 # 10.02 # 23.01 # 27.74 # ...,17.19,...,-2.62,-2.62,175.0,10,2,33.83,-3.54,2006-02-06,28,75
3,AMZN,8,75.00,21.08,334.0,16.77,3.458111,-0.5 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 21.88,33.87,...,-17.29,-17.29,261.0,6,2,70.00,-34.09,2006-02-06,28,75
4,NVDA,9,66.67,18.53,367.0,28.37,3.113139,66.11 # -39.26 # 29.65 # -14.24 # 1.59 # 34.55...,66.11 # 29.65 # 1.59 # 34.55 # 28.37 # 63.04,37.22,...,-18.83,-14.24,395.0,6,3,66.11,-39.26,2006-05-19,28,75
5,META,8,87.50,18.50,173.0,23.57,3.196840,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,26.35,...,-36.41,-36.41,105.0,7,1,39.95,-36.41,2012-07-30,28,75
6,TSLA,11,81.82,39.31,180.0,18.14,22.072490,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,48.98,...,-4.21,-4.21,134.0,9,2,154.55,-7.21,2012-01-17,28,75
7,LLY,12,91.67,11.11,217.0,11.99,3.144840,3.5 # -28.71 # 11.61 # 10.66 # 16.32 # 10.44 #...,3.5 # 11.61 # 10.66 # 16.32 # 10.44 # 12.37 # ...,14.73,...,-28.71,-28.71,168.0,11,1,35.88,-28.71,2006-02-06,28,75
8,V,7,100.00,34.70,474.0,27.32,7.441021,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,34.70,...,NaN,NaN,439.0,7,0,81.51,14.64,2008-10-08,28,75
9,AVGO,6,100.00,42.19,388.0,41.70,7.859202,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,42.19,...,NaN,NaN,306.0,6,0,72.17,13.79,2011-08-09,28,75


In [7]:
tradesdf

,ticker,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,buy_open,buy_low,buy_high,...,sell_sma_100,sell_diff_sma100,sell_sma_200,sell_diff_sma200,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_local,sell_local_text
0,AAPL,1.235629,2.572143,3.178214,2006-02-06,2006-11-22,289,2.572143,2.383571,2.589643,...,2.552104,26.380336,2.430057,32.727623,2.991023,3.249548,0.750016,-7.265357,,
1,AAPL,1.319094,4.999643,6.595000,2008-01-24,2008-05-06,103,4.999643,4.714643,5.025000,...,5.428157,22.812013,5.498979,21.230315,6.159912,6.786946,1.807812,-7.598031,,
2,AAPL,0.984237,4.780714,4.705357,2008-09-16,2009-05-05,231,4.780714,4.719643,5.089286,...,3.561411,33.083306,4.053750,16.919965,4.365946,4.819626,1.687520,-7.884491,,
3,AAPL,0.885468,20.022499,17.729286,2012-11-08,2013-08-15,280,20.022499,19.117500,20.079643,...,15.554496,14.323851,16.760509,6.097610,15.938828,18.458173,3.799653,-10.367900,,
4,AAPL,1.077202,28.237499,30.417500,2015-08-05,2017-01-26,540,28.237499,28.025000,29.360001,...,28.479700,7.041158,26.786475,13.807437,29.729285,30.657715,0.566556,-2.478975,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,MRK,1.163603,51.383587,59.790077,2014-10-17,2016-08-08,661,51.383587,51.078243,52.127865,...,53.764599,11.562098,51.819227,15.750308,52.234347,62.475578,4.159094,-12.915056,,
176,MRK,1.028737,57.776718,59.437023,2017-10-27,2018-01-17,82,57.776718,55.171757,57.776718,...,57.028149,3.788974,58.777242,0.700421,52.788061,60.822625,2.760134,-10.814301,,
177,MRK,1.183224,74.055344,87.624046,2019-04-17,2019-12-23,250,74.055344,70.372139,74.064888,...,81.314504,7.653820,79.357729,10.308310,84.182510,88.317493,0.890267,-3.833368,,
178,MRK,1.092927,77.013359,84.169998,2020-02-27,2021-10-04,585,77.013359,74.427483,78.005722,...,75.882383,9.511583,74.915189,10.925433,69.831816,85.376182,2.739090,-15.966525,,


# EDA on results

In [ ]:
from IPython.display import display
display(tradesdf)
display(trades_summarydf)


,ticker,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,buy_open,buy_low,buy_high,...,sell_sma_100,sell_diff_sma100,sell_sma_200,sell_diff_sma200,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_local,sell_local_text
0,AAPL,1.365296,2.327857,3.178214,2006-02-10,2006-11-22,285,2.327857,2.246429,2.416786,...,2.552104,26.380336,2.430057,32.727623,2.991023,3.249548,0.750016,-7.265357,,
1,AAPL,1.319094,4.999643,6.595000,2008-01-24,2008-05-06,103,4.999643,4.714643,5.025000,...,5.428157,22.812013,5.498979,21.230315,6.159912,6.786946,1.807812,-7.598031,,
2,AAPL,0.984237,4.780714,4.705357,2008-09-16,2009-05-05,231,4.780714,4.719643,5.089286,...,3.561411,33.083306,4.053750,16.919965,4.365946,4.819626,1.687520,-7.884491,,
3,AAPL,0.885468,20.022499,17.729286,2012-11-08,2013-08-15,280,20.022499,19.117500,20.079643,...,15.554496,14.323851,16.760509,6.097610,15.938828,18.458173,3.799653,-10.367900,,
4,AAPL,1.077202,28.237499,30.417500,2015-08-05,2017-01-26,540,28.237499,28.025000,29.360001,...,28.479700,7.041158,26.786475,13.807437,29.729285,30.657715,0.566556,-2.478975,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,COST,1.067977,219.339996,234.250000,2018-10-08,2019-03-13,156,219.339996,219.259995,224.500000,...,218.331400,8.266610,219.291650,7.792524,215.439543,242.016463,2.384490,-8.858813,,
183,COST,1.161051,294.440002,341.859985,2020-03-02,2020-08-05,156,294.440002,291.380005,309.510010,...,307.750700,10.469286,304.925800,11.492698,318.477478,345.294532,1.566177,-6.321888,,
184,COST,1.234062,335.510010,414.040009,2021-02-26,2021-07-12,136,335.510010,330.940002,335.890015,...,366.998300,11.139481,367.703050,10.926468,397.334554,415.041435,1.755769,-2.585430,,
185,COST,1.237788,489.890015,606.380005,2022-01-19,2022-04-08,79,489.890015,487.100006,496.239990,...,535.099899,12.136066,492.778599,21.766647,562.110862,615.221120,2.530022,-6.321098,,


,ticker,number_of_trades,win_ratio(%),average_res(%),average_trade_len(days),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,...,looser_trades_mean,looser_trades_median,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
0,AAPL,9,77.78,16.23,224.0,7.72,3.479083,36.53 # 31.91 # -1.58 # -11.45 # 7.72 # 30.13 ...,36.53 # 31.91 # 7.72 # 30.13 # 6.63 # 41.03 # ...,22.73,...,-6.51,-6.51,179.0,7,2,41.03,-11.45,2006-02-10,28,75
1,MSFT,9,77.78,7.82,242.0,14.39,1.825421,5.51 # 14.39 # -21.79 # -5.34 # 14.67 # 4.79 #...,5.51 # 14.39 # 14.67 # 4.79 # 15.39 # 23.4 # 19.4,13.94,...,-13.57,-13.57,170.0,7,2,23.40,-21.79,2006-05-01,28,75
2,GOOG,12,83.33,13.89,191.0,13.20,4.505082,19.98 # 6.74 # 5.18 # -1.7 # 10.02 # 23.01 # 2...,19.98 # 6.74 # 5.18 # 10.02 # 23.01 # 27.74 # ...,17.19,...,-2.62,-2.62,175.0,10,2,33.83,-3.54,2006-02-06,28,75
3,AMZN,8,87.50,21.29,335.0,16.77,3.516816,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,29.20,...,-34.09,-34.09,262.5,7,1,70.00,-34.09,2006-02-03,28,75
4,NVDA,9,66.67,18.53,367.0,28.37,3.113139,66.11 # -39.26 # 29.65 # -14.24 # 1.59 # 34.55...,66.11 # 29.65 # 1.59 # 34.55 # 28.37 # 63.04,37.22,...,-18.83,-14.24,395.0,6,3,66.11,-39.26,2006-05-19,28,75
5,META,8,87.50,18.50,173.0,23.57,3.196840,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,26.35,...,-36.41,-36.41,105.0,7,1,39.95,-36.41,2012-07-30,28,75
6,TSLA,11,90.91,40.08,180.0,18.14,23.948082,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,44.81,...,-7.21,-7.21,134.0,10,1,154.55,-7.21,2012-01-17,28,75
7,LLY,12,91.67,11.19,217.0,11.99,3.173300,4.43 # -28.71 # 11.61 # 10.66 # 16.32 # 10.44 ...,4.43 # 11.61 # 10.66 # 16.32 # 10.44 # 12.37 #...,14.81,...,-28.71,-28.71,168.0,11,1,35.88,-28.71,2006-02-07,28,75
8,V,7,100.00,34.70,474.0,27.32,7.441021,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,34.70,...,NaN,NaN,439.0,7,0,81.51,14.64,2008-10-08,28,75
9,JPM,11,90.91,10.71,303.0,9.13,2.948838,9.13 # -4.11 # 6.65 # 16.23 # 11.05 # 35.78 # ...,9.13 # 6.65 # 16.23 # 11.05 # 35.78 # 10.28 # ...,12.19,...,-4.11,-4.11,197.0,10,1,35.78,-4.11,2007-03-06,28,75


In [ ]:
tradesdf['result'].mean()

1.1651367319013604

0      1.157554
1      1.120946
2      0.709784
3      0.801907
4      0.888815
         ...   
294    1.095340
295    1.155336
296    1.150670
297    0.954237
298    1.045829
Name: result, Length: 299, dtype: float64

In [ ]:
# profitable trades
round(((sum(tradesdf['result'] >1) / tradesdf.shape[0])*100),2)

88.24

In [ ]:
trades_summarydf.sort_values(by='median_res(%)', ascending=False)

,ticker,number_of_trades,win_ratio(%),average_res(%),average_trade_len(days),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,...,looser_trades_mean,looser_trades_median,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
11,AVGO,6,100.00,42.19,388.0,41.70,7.859202,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,42.19,...,NaN,NaN,306.0,6,0,72.17,13.79,2011-08-09,28,75
4,NVDA,9,66.67,18.53,367.0,28.37,3.113139,66.11 # -39.26 # 29.65 # -14.24 # 1.59 # 34.55...,66.11 # 29.65 # 1.59 # 34.55 # 28.37 # 63.04,37.22,...,-18.83,-14.24,395.0,6,3,66.11,-39.26,2006-05-19,28,75
8,V,7,100.00,34.70,474.0,27.32,7.441021,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,34.70,...,NaN,NaN,439.0,7,0,81.51,14.64,2008-10-08,28,75
14,MA,6,100.00,25.79,411.0,26.15,3.861115,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,25.79,...,NaN,NaN,246.0,6,0,43.05,6.67,2010-02-05,28,75
5,META,8,87.50,18.50,173.0,23.57,3.196840,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,26.35,...,-36.41,-36.41,105.0,7,1,39.95,-36.41,2012-07-30,28,75
6,TSLA,11,90.91,40.08,180.0,18.14,23.948082,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,44.81,...,-7.21,-7.21,134.0,10,1,154.55,-7.21,2012-01-17,28,75
3,AMZN,8,87.50,21.29,335.0,16.77,3.516816,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,29.20,...,-34.09,-34.09,262.5,7,1,70.00,-34.09,2006-02-03,28,75
19,COST,11,100.00,15.84,213.0,16.11,4.863675,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,15.84,...,NaN,NaN,164.0,11,0,31.69,2.92,2008-10-10,28,75
1,MSFT,9,77.78,7.82,242.0,14.39,1.825421,5.51 # 14.39 # -21.79 # -5.34 # 14.67 # 4.79 #...,5.51 # 14.39 # 14.67 # 4.79 # 15.39 # 23.4 # 19.4,13.94,...,-13.57,-13.57,170.0,7,2,23.40,-21.79,2006-05-01,28,75
17,HD,9,77.78,11.17,343.0,14.17,2.331472,-20.67 # 18.29 # 29.61 # 31.57 # 14.17 # 23.57...,18.29 # 29.61 # 31.57 # 14.17 # 23.57 # 4.21 #...,18.48,...,-14.42,-14.42,202.0,7,2,31.57,-20.67,2006-07-12,28,75


In [ ]:
trades_summarydf.sort_values(by='average_res(%)', ascending=False)

,ticker,number_of_trades,win_ratio(%),average_res(%),average_trade_len(days),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,...,looser_trades_mean,looser_trades_median,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
11,AVGO,6,100.00,42.19,388.0,41.70,7.859202,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,42.19,...,NaN,NaN,306.0,6,0,72.17,13.79,2011-08-09,28,75
6,TSLA,11,90.91,40.08,180.0,18.14,23.948082,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,44.81,...,-7.21,-7.21,134.0,10,1,154.55,-7.21,2012-01-17,28,75
8,V,7,100.00,34.70,474.0,27.32,7.441021,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,34.70,...,NaN,NaN,439.0,7,0,81.51,14.64,2008-10-08,28,75
14,MA,6,100.00,25.79,411.0,26.15,3.861115,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,25.79,...,NaN,NaN,246.0,6,0,43.05,6.67,2010-02-05,28,75
3,AMZN,8,87.50,21.29,335.0,16.77,3.516816,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,29.20,...,-34.09,-34.09,262.5,7,1,70.00,-34.09,2006-02-03,28,75
4,NVDA,9,66.67,18.53,367.0,28.37,3.113139,66.11 # -39.26 # 29.65 # -14.24 # 1.59 # 34.55...,66.11 # 29.65 # 1.59 # 34.55 # 28.37 # 63.04,37.22,...,-18.83,-14.24,395.0,6,3,66.11,-39.26,2006-05-19,28,75
5,META,8,87.50,18.50,173.0,23.57,3.196840,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,26.35,...,-36.41,-36.41,105.0,7,1,39.95,-36.41,2012-07-30,28,75
0,AAPL,9,77.78,16.23,224.0,7.72,3.479083,36.53 # 31.91 # -1.58 # -11.45 # 7.72 # 30.13 ...,36.53 # 31.91 # 7.72 # 30.13 # 6.63 # 41.03 # ...,22.73,...,-6.51,-6.51,179.0,7,2,41.03,-11.45,2006-02-10,28,75
19,COST,11,100.00,15.84,213.0,16.11,4.863675,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,15.84,...,NaN,NaN,164.0,11,0,31.69,2.92,2008-10-10,28,75
12,WMT,13,100.00,15.35,284.0,11.37,6.076947,9.15 # 2.51 # 14.19 # 25.04 # 10.15 # 9.03 # 1...,9.15 # 2.51 # 14.19 # 25.04 # 10.15 # 9.03 # 1...,15.35,...,NaN,NaN,246.0,13,0,42.43,2.51,2006-07-14,28,75


In [ ]:
trades_summarydf.sort_values(by='cumulative_result', ascending=False)

,ticker,number_of_trades,win_ratio(%),average_res(%),average_trade_len(days),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,...,looser_trades_mean,looser_trades_median,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
6,TSLA,11,90.91,40.08,180.0,18.14,23.948082,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,44.81,...,-7.21,-7.21,134.0,10,1,154.55,-7.21,2012-01-17,28,75
11,AVGO,6,100.00,42.19,388.0,41.70,7.859202,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,42.19,...,NaN,NaN,306.0,6,0,72.17,13.79,2011-08-09,28,75
8,V,7,100.00,34.70,474.0,27.32,7.441021,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,34.70,...,NaN,NaN,439.0,7,0,81.51,14.64,2008-10-08,28,75
12,WMT,13,100.00,15.35,284.0,11.37,6.076947,9.15 # 2.51 # 14.19 # 25.04 # 10.15 # 9.03 # 1...,9.15 # 2.51 # 14.19 # 25.04 # 10.15 # 9.03 # 1...,15.35,...,NaN,NaN,246.0,13,0,42.43,2.51,2006-07-14,28,75
19,COST,11,100.00,15.84,213.0,16.11,4.863675,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,15.84,...,NaN,NaN,164.0,11,0,31.69,2.92,2008-10-10,28,75
2,GOOG,12,83.33,13.89,191.0,13.20,4.505082,19.98 # 6.74 # 5.18 # -1.7 # 10.02 # 23.01 # 2...,19.98 # 6.74 # 5.18 # 10.02 # 23.01 # 27.74 # ...,17.19,...,-2.62,-2.62,175.0,10,2,33.83,-3.54,2006-02-06,28,75
14,MA,6,100.00,25.79,411.0,26.15,3.861115,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,25.79,...,NaN,NaN,246.0,6,0,43.05,6.67,2010-02-05,28,75
3,AMZN,8,87.50,21.29,335.0,16.77,3.516816,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,29.20,...,-34.09,-34.09,262.5,7,1,70.00,-34.09,2006-02-03,28,75
0,AAPL,9,77.78,16.23,224.0,7.72,3.479083,36.53 # 31.91 # -1.58 # -11.45 # 7.72 # 30.13 ...,36.53 # 31.91 # 7.72 # 30.13 # 6.63 # 41.03 # ...,22.73,...,-6.51,-6.51,179.0,7,2,41.03,-11.45,2006-02-10,28,75
5,META,8,87.50,18.50,173.0,23.57,3.196840,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,26.35,...,-36.41,-36.41,105.0,7,1,39.95,-36.41,2012-07-30,28,75


In [ ]:
trades_summarydf.sort_values(by='win_ratio(%)', ascending=False)

,ticker,number_of_trades,win_ratio(%),average_res(%),average_trade_len(days),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,...,looser_trades_mean,looser_trades_median,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
19,COST,11,100.00,15.84,213.0,16.11,4.863675,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,2.92 # 24.66 # 7.61 # 21.72 # 4.79 # 10.73 # 6...,15.84,...,NaN,NaN,164.0,11,0,31.69,2.92,2008-10-10,28,75
8,V,7,100.00,34.70,474.0,27.32,7.441021,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,34.70,...,NaN,NaN,439.0,7,0,81.51,14.64,2008-10-08,28,75
11,AVGO,6,100.00,42.19,388.0,41.70,7.859202,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,53.39 # 46.67 # 30.41 # 13.79 # 72.17 # 36.73,42.19,...,NaN,NaN,306.0,6,0,72.17,13.79,2011-08-09,28,75
12,WMT,13,100.00,15.35,284.0,11.37,6.076947,9.15 # 2.51 # 14.19 # 25.04 # 10.15 # 9.03 # 1...,9.15 # 2.51 # 14.19 # 25.04 # 10.15 # 9.03 # 1...,15.35,...,NaN,NaN,246.0,13,0,42.43,2.51,2006-07-14,28,75
14,MA,6,100.00,25.79,411.0,26.15,3.861115,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,6.67 # 34.46 # 18.24 # 25.71 # 26.6 # 43.05,25.79,...,NaN,NaN,246.0,6,0,43.05,6.67,2010-02-05,28,75
7,LLY,12,91.67,11.19,217.0,11.99,3.173300,4.43 # -28.71 # 11.61 # 10.66 # 16.32 # 10.44 ...,4.43 # 11.61 # 10.66 # 16.32 # 10.44 # 12.37 #...,14.81,...,-28.71,-28.71,168.0,11,1,35.88,-28.71,2006-02-07,28,75
15,JNJ,12,91.67,10.05,318.0,9.06,3.042926,10.13 # 1.87 # -3.23 # 6.71 # 9.62 # 9.28 # 30...,10.13 # 1.87 # 6.71 # 9.62 # 9.28 # 30.21 # 9....,11.26,...,-3.23,-3.23,280.5,11,1,30.21,-3.23,2006-01-30,28,75
16,PG,11,90.91,10.18,426.0,10.05,2.818501,9.91 # 10.91 # -5.86 # 9.26 # 10.05 # 22.23 # ...,9.91 # 10.91 # 9.26 # 10.05 # 22.23 # 19.37 # ...,11.78,...,-5.86,-5.86,208.0,10,1,22.23,-5.86,2006-04-13,28,75
6,TSLA,11,90.91,40.08,180.0,18.14,23.948082,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,44.81,...,-7.21,-7.21,134.0,10,1,154.55,-7.21,2012-01-17,28,75
9,JPM,11,90.91,10.71,303.0,9.13,2.948838,9.13 # -4.11 # 6.65 # 16.23 # 11.05 # 35.78 # ...,9.13 # 6.65 # 16.23 # 11.05 # 35.78 # 10.28 # ...,12.19,...,-4.11,-4.11,197.0,10,1,35.78,-4.11,2007-03-06,28,75


In [ ]:
tradesdf

,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,trade_id,status,buy_open,buy_low,...,sell_sma_100,sell_diff_sma100,sell_sma_200,sell_diff_sma200,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_local,sell_local_text
0,1.157554,2.096786,2.427143,2006-03-28,2006-07-31,125,1,closed,2.129643,2.080357,...,2.235796,8.558323,NaN,NaN,2.157726,2.452988,1.064834,-11.100162,,
1,1.120946,4.966786,5.567500,2008-01-23,2008-04-07,75,2,closed,4.863929,4.505000,...,5.515200,0.948290,5.370007,3.677704,5.205051,5.617806,0.903558,-6.510080,,
2,0.709784,5.417143,3.845000,2008-09-09,2009-03-23,195,3,closed,5.602143,5.349643,...,3.318079,15.880320,4.356388,-11.738797,3.462633,3.852367,0.191591,-9.944517,,
3,0.801907,20.600000,16.519285,2012-11-02,2013-08-02,273,4,closed,21.281786,20.526787,...,15.474689,6.750352,17.006387,-2.864231,15.885276,16.584153,0.392677,-3.837996,,
4,0.888815,30.017500,26.680000,2015-07-09,2016-03-22,257,5,closed,30.962500,29.805000,...,26.540700,0.524856,28.026800,-4.805399,26.349669,26.682331,0.008737,-1.238121,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1.095340,46.779999,51.240002,2018-10-11,2019-02-05,117,9,closed,47.330002,46.490002,...,48.420500,5.822950,47.776300,7.249833,49.962860,51.505140,0.517445,-2.492469,,
295,1.155336,50.599998,58.459999,2019-05-31,2019-07-02,32,10,closed,51.240002,50.509998,...,53.634000,8.998022,51.043150,14.530547,55.925769,58.822230,0.619622,-4.334981,,
296,1.150670,53.759998,61.860001,2019-08-05,2020-12-15,498,11,closed,55.099998,53.689999,...,57.796300,7.031074,55.053950,12.362511,58.799952,62.176048,0.510907,-4.946733,,
297,0.954237,81.070000,77.360001,2022-01-25,2022-10-28,276,12,closed,81.330002,80.199997,...,71.427600,8.305473,74.500150,3.838718,71.466509,77.889491,0.684450,-7.618267,,
